In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
# fetch "New York City" page from Wikipedia
from pathlib import Path

import requests
response = requests.get(
    'https://en.wikipedia.org/w/api.php',
    params={
        'action': 'query',
        'format': 'json',
        'titles': 'New York City',
        'prop': 'extracts',
        # 'exintro': True,
        'explaintext': True,
    }
).json()
page = next(iter(response['query']['pages'].values()))
nyc_text = page['extract']

data_path = Path('data')
if not data_path.exists():
    Path.mkdir(data_path)

with open('data/nyc_text.txt', 'w') as fp:
    fp.write(nyc_text)

In [ ]:
# My OpenAI Key
import os
os.environ['OPENAI_API_KEY'] = "INSERT OPENAI KEY"

In [18]:
from gpt_index import GPTSimpleVectorIndex, SimpleDirectoryReader

In [5]:
documents = SimpleDirectoryReader('data').load_data()
index = GPTSimpleVectorIndex.from_documents(documents)

INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:gpt_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 28324 tokens
> [build_index_from_nodes] Total embedding token usage: 28324 tokens


In [6]:
index.save_to_disk('index.json')

In [7]:
import json

In [8]:
with open('index.json', 'r') as f:
    save_dict = json.load(f)

In [30]:
save_dict['index_struct']['__data__']['nodes_dict'] = {}

In [31]:
save_dict

{'index_struct': {'__type__': 'simple_dict',
  '__data__': {'index_id': 'e93049db-89b2-42f4-8a68-b403cd94e3ff',
   'nodes_dict': {},
   'doc_id_dict': {'a19ffc39-5f94-41bf-a4e1-bd41b425710f': ['e2ad1386-0f2f-46fc-98a1-913e46c8adc8',
     'fcb5c5aa-3dda-4749-b939-c3c05023d92b',
     'a233266b-5bf1-419c-a593-de9ec95eb346',
     '81c54c78-bd8e-4417-9a71-9f3c5d0c02b5',
     '59368f90-d5f8-4f7d-9137-f18efc09f2b6',
     '80633009-ffee-4fee-8ba6-9c54a7a29ada',
     '51fffec8-bd54-4bac-9a9c-a4b98f37c231',
     'a6262ff2-8813-48f6-b25d-fcfaba75d456',
     'dab1616f-926c-473c-954e-53389a55a560']},
   'embeddings_dict': {}}},
 'docstore': {'docs': {'4776b927-1367-4a49-bfb9-dc0b634e5bd2': {'text': 'New York, often called New York City or NYC, is the most populous city in the United States. With a 2020 population of 8,804,190 distributed over 300.46 square miles (778.2 km2), New York City is the most densely populated major city in the United States and more than twice as populous as Los Angeles, t

In [32]:
index = GPTSimpleVectorIndex.load_from_dict(save_dict)

<class 'gpt_index.data_structs.data_structs.SimpleIndexDict'>
dict_keys(['index_id', 'nodes_dict', 'doc_id_dict', 'embeddings_dict'])


AssertionError: 

In [ ]:
# try loading
new_index = GPTTreeIndex.load_from_disk('index.json')

In [ ]:
# GPT doesn't find the corresponding evidence in the leaf node, but still gives the correct answer
# set Logging to DEBUG for more detailed outputs

new_index.query("What is the name of the professional women's basketball team in New York City?")

In [ ]:
# GPT doesn't find the corresponding evidence in the leaf node, but still gives the correct answer
# set Logging to DEBUG for more detailed outputs

new_index.query("What battles took place in New York City in the American Revolution?")

In [ ]:
# GPT doesn't find the corresponding evidence in the leaf node, but still gives the correct answer
# set Logging to DEBUG for more detailed outputs

new_index.query("What are the airports in New York City?")

In [ ]:
# Try using embedding query
new_index.query("What are the airports in New York City?", mode="embedding")